In [ ]:
这个是lgb代码，用于生成merge.ipynb所需要的结果文件finallgb.txt
其中feature_answer_base2.h5文件缺失，需要运行一次feature_final_answer_base.ipynb的文件生成

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
import gc

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
target=train['target']

In [3]:
u7days_invite_count=pd.read_hdf('q7days_invite_count2.h5', key='data')#######################
u7days_invite_count.columns=['qid','day','q7days_invite_pcount','q7days_invite_daynunique']
u7days_invite_count['day']=u7days_invite_count['day'].astype(str)
train['day']=train['time'].apply(lambda x:x[1:].split('-H')[0])
test['day']=test['time'].apply(lambda x:x[1:].split('-H')[0])

train=pd.merge(train,u7days_invite_count,how='left',on=['qid','day'])
test=pd.merge(test,u7days_invite_count,how='left',on=['qid','day'])

train.drop(['time','day'],axis=1,inplace=True)
test.drop(['time','day'],axis=1,inplace=True)

In [4]:
feature_topic_count=pd.read_hdf('feature_topic_count.h5', key='data')
train=pd.concat([train,feature_topic_count.iloc[0:train.shape[0],:]],axis=1)
test=pd.concat([test,feature_topic_count.iloc[train.shape[0]:,:].reset_index(drop=True)],axis=1)

In [5]:
feature_topic_tfidf=pd.read_hdf('feature_topic_tfidf.h5', key='data')
feature_topic_w2vdis=pd.read_hdf('feature_topic_w2vdis.h5', key='data').reset_index(drop=True)
feature_uid_ctr=pd.read_hdf('feature_uid_ctr_new.h5', key='data').reset_index(drop=True)
feature_member_info=pd.read_hdf('feature_member_info_new.h5', key='data').reset_index(drop=True)
feature_invite_time_new=pd.read_hdf('feature_invite_time_new2.h5', key='data').reset_index(drop=True)#######################
feature_qidtop_tfidf=pd.read_hdf('feature_qidtop_tfidf.h5', key='data')
feature_uq_pagerank=pd.read_hdf('feature_uq_pagerank2.h5', key='data')#######################
feature_today_invite=pd.read_hdf('feature_today_invite_new2.h5', key='data').reset_index(drop=True)#######################

model3=pd.read_hdf('stackingesim_wnf.h5', key='data').reset_index(drop=True)
model4=pd.read_hdf('stackingesim_wnfchar.h5', key='data').reset_index(drop=True)
model5=pd.read_hdf('stackinglstur_wnf.h5', key='data').reset_index(drop=True)

train=pd.concat([train,
                 feature_topic_tfidf.iloc[0:train.shape[0],:],
                 feature_topic_w2vdis.iloc[0:train.shape[0],:],
                 feature_uid_ctr.iloc[0:train.shape[0],:],
                 feature_member_info.iloc[0:train.shape[0],:],
                 feature_invite_time_new.iloc[0:train.shape[0],3:],
                 feature_qidtop_tfidf.iloc[0:train.shape[0],:],
                 feature_uq_pagerank.iloc[0:train.shape[0],:],
                 feature_today_invite.iloc[0:train.shape[0],:],
                 model3.iloc[0:train.shape[0],:],
                 model4.iloc[0:train.shape[0],:],
                 model5.iloc[0:train.shape[0],:],

                 
                ],axis=1)
test=pd.concat([test,
                feature_topic_tfidf.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_topic_w2vdis.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_uid_ctr.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_member_info.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_invite_time_new.iloc[train.shape[0]:,3:].reset_index(drop=True),
                feature_qidtop_tfidf.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_uq_pagerank.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_today_invite.iloc[train.shape[0]:,:].reset_index(drop=True),
                model3.iloc[train.shape[0]:,:].reset_index(drop=True),
                model4.iloc[train.shape[0]:,:].reset_index(drop=True),
                model5.iloc[train.shape[0]:,:].reset_index(drop=True),
               ],axis=1)

In [6]:
del feature_topic_tfidf,feature_topic_w2vdis,feature_uid_ctr,feature_member_info,feature_qidtop_tfidf,feature_today_invite,feature_invite_time_new
del feature_topic_count,u7days_invite_count,model3,model4,model5,feature_uq_pagerank
gc.collect()#feature_invite_timefeature_uq_pagerank


90

In [7]:
feature_topic_answer=pd.read_hdf('feature_topic_answer.h5', key='data')
train=pd.merge(train,feature_topic_answer,how='left',on='qid')
test=pd.merge(test,feature_topic_answer,how='left',on='qid')

feature_topicn_tongji=pd.read_hdf('feature_topicn_tongji.h5', key='data')
train=pd.merge(train,feature_topicn_tongji,how='left',on='uid')
test=pd.merge(test,feature_topicn_tongji,how='left',on='uid')

feature_uid_embed=pd.read_hdf('feature_uid_embed.h5', key='data')
train=pd.merge(train,feature_uid_embed,how='left',on='uid')
test=pd.merge(test,feature_uid_embed,how='left',on='uid')


feature_weight_vector=pd.read_hdf('feature_weight_vector.h5', key='data')
train=pd.merge(train,feature_weight_vector,how='left',on='uid')
test=pd.merge(test,feature_weight_vector,how='left',on='uid')

In [8]:
del feature_topic_answer,feature_topicn_tongji,feature_uid_embed,feature_weight_vector
gc.collect()

272

In [9]:
feature_qtitle_bm25=pd.read_hdf('feature_qtitle_bm25new.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_qtitle_bm25,how='left',on=['qid','uid'])
test=pd.merge(test,feature_qtitle_bm25,how='left',on=['qid','uid'])


feature_topic_meta=pd.read_hdf('feature_topic_meta_new.h5', key='data')
train=pd.merge(train,feature_topic_meta,how='left',on=['qid','uid'])
test=pd.merge(test,feature_topic_meta,how='left',on=['qid','uid'])

feature_topic_n_stacking=pd.read_hdf('feature_topic_n_stacking.h5', key='data')
train=pd.merge(train,feature_topic_n_stacking,how='left',on=['qid','uid'])
test=pd.merge(test,feature_topic_n_stacking,how='left',on=['qid','uid'])

feature_answer_base=pd.read_hdf('feature_answer_base2.h5', key='data')
train=pd.merge(train,feature_answer_base,how='left',on=['qid','uid'])
test=pd.merge(test,feature_answer_base,how='left',on=['qid','uid'])

feature_qid_freq=pd.read_hdf('feature_qid_freq2.h5', key='data').drop_duplicates(['qid','uid'])#已校验
train=pd.merge(train,feature_qid_freq,how='left',on=['qid','uid'])
test=pd.merge(test,feature_qid_freq,how='left',on=['qid','uid'])

feature_uid_freq=pd.read_hdf('feature_uid_freq2.h5', key='data').drop_duplicates(['qid','uid'])#已校验
train=pd.merge(train,feature_uid_freq,how='left',on=['qid','uid'])
test=pd.merge(test,feature_uid_freq,how='left',on=['qid','uid'])

In [10]:
feature_qa_count=pd.read_hdf('feature_qa_count.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_qa_count,how='left',on=['qid','uid'])
test=pd.merge(test,feature_qa_count,how='left',on=['qid','uid'])


feature_ua_count=pd.read_hdf('feature_ua_count.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_ua_count,how='left',on=['qid','uid'])
test=pd.merge(test,feature_ua_count,how='left',on=['qid','uid'])

##########################################################80
del feature_qtitle_bm25,feature_topic_meta,feature_topic_n_stacking,feature_answer_base#,feature_qa_count,feature_ua_count
gc.collect()

feature_Stime_history=pd.read_hdf('feature_Stime_history.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_Stime_history,how='left',on=['qid','uid'])
test=pd.merge(test,feature_Stime_history,how='left',on=['qid','uid'])

feature_yesterday_invite=pd.read_hdf('feature_yesterday_invite2.h5', key='data').drop_duplicates(['qid','uid'])########
train=pd.merge(train,feature_yesterday_invite,how='left',on=['qid','uid'])
test=pd.merge(test,feature_yesterday_invite,how='left',on=['qid','uid'])

feature_qid_topic_uid_count=pd.read_hdf('feature_qid_topic_uid_count.h5', key='data')
train=pd.merge(train,feature_qid_topic_uid_count,how='left',on='qid')
test=pd.merge(test,feature_qid_topic_uid_count,how='left',on='qid')



In [11]:
feature_uid_topic_qid_count=pd.read_hdf('feature_uid_topic_qid_count.h5', key='data')
train=pd.merge(train,feature_uid_topic_qid_count,how='left',on='uid')
test=pd.merge(test,feature_uid_topic_qid_count,how='left',on='uid')

feature_atime_new=pd.read_hdf('feature_atime_new.h5', key='data')
train=pd.merge(train,feature_atime_new,how='left',on=['qid','uid'])
test=pd.merge(test,feature_atime_new,how='left',on=['qid','uid'])

u7days_invite_count=pd.read_hdf('u7days_invite_count2.h5', key='data')
train=pd.merge(train,u7days_invite_count,how='left',on=['qid','uid'])
test=pd.merge(test,u7days_invite_count,how='left',on=['qid','uid'])

del feature_Stime_history,feature_yesterday_invite,feature_qid_topic_uid_count,feature_uid_topic_qid_count,feature_atime_new
del u7days_invite_count
gc.collect()

126

In [12]:
feature_intime_gap=pd.read_hdf('feature_intime_gap2.h5', key='data')###################
train=pd.merge(train,feature_intime_gap,how='left',on='uid')
test=pd.merge(test,feature_intime_gap,how='left',on='uid')

feature_qintime_gap=pd.read_hdf('feature_qintime_gap2.h5', key='data')#################
train=pd.merge(train,feature_qintime_gap,how='left',on='qid')
test=pd.merge(test,feature_qintime_gap,how='left',on='qid')

In [13]:
del feature_intime_gap,feature_qintime_gap
gc.collect()

62

In [14]:
len(train)+len(test)

10630880

In [15]:
train.shape

(9489162, 331)

In [16]:
test.shape

(1141718, 330)

In [17]:
test = test.drop(['uid','qid'], axis=1)
train = train.drop(['target', 'uid','qid'], axis=1)

In [18]:
test.shape

(1141718, 328)

In [19]:
train.shape

(9489162, 328)

In [20]:
param = {'num_leaves': 100,
         'objective':'binary',
         'learning_rate': 0.05,        
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1,
         "num_threads" : -1,
#          'device': 'gpu',
#          'gpu_platform_id': 0,
#          'gpu_device_id': 0
         }

In [21]:
cat_col=['sex','look_freq','a','b','c','d','e','A','B','C','D','E']

In [22]:
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,RepeatedKFold
#folds = GroupKFold(n_splits=5)
folds = KFold(n_splits=5, shuffle=True, random_state=15)
feature_importance_df = pd.DataFrame()
predictions=np.zeros([len(test),1])
stack_train = np.zeros((len(train),1))
stack_test = np.zeros((len(test),1))

In [23]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split( target.values)):
#for fold_, (trn_idx, val_idx) in enumerate(folds.split(stack_train, target,groups=uid['uid'])):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx],
                           label=target.iloc[trn_idx],
                           categorical_feature=cat_col
                           )
    val_data = lgb.Dataset(train.iloc[val_idx],
                           label=target.iloc[val_idx],
                           categorical_feature=cat_col
                           )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds=100)
    stack_train[val_idx, :] =clf.predict(train.iloc[val_idx], num_iteration=clf.best_iteration).reshape(-1,1)
    stack_test += clf.predict(test, num_iteration=clf.best_iteration).reshape(-1,1)
    if fold_==1:
        break

fold n°0


/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.872295	valid_1's auc: 0.87151
[200]	training's auc: 0.881837	valid_1's auc: 0.879526
[300]	training's auc: 0.887201	valid_1's auc: 0.882727
[400]	training's auc: 0.891128	valid_1's auc: 0.884522
[500]	training's auc: 0.894225	valid_1's auc: 0.88561
[600]	training's auc: 0.896841	valid_1's auc: 0.886417
[700]	training's auc: 0.899061	valid_1's auc: 0.886997
[800]	training's auc: 0.901049	valid_1's auc: 0.887506
[900]	training's auc: 0.902874	valid_1's auc: 0.88791
[1000]	training's auc: 0.904449	valid_1's auc: 0.888166
[1100]	training's auc: 0.905847	valid_1's auc: 0.888367
[1200]	training's auc: 0.907206	valid_1's auc: 0.88858
[1300]	training's auc: 0.908469	valid_1's auc: 0.888729
[1400]	training's auc: 0.909593	valid_1's auc: 0.88886
[1500]	training's auc: 0.910773	valid_1's auc: 0.888989
[1600]	training's auc: 0.911841	valid_1's auc: 0.889129
[1700]	training's auc: 0.912949	valid_1's auc: 0.889232


KeyboardInterrupt: 

In [25]:
stack_test/=2

In [27]:
stack_test.mean()

0.16565486133338583

In [28]:
test_id=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
test_id['target']=stack_test
test_id.to_csv('finallgb.txt',index=False, sep='\t',header=None)